## Import Libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from torchmetrics.functional.classification import binary_accuracy, binary_auroc, binary_f1_score, binary_precision, binary_recall
import tensorflow as tf
import time
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, InputSpec
from keras.utils import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *

## Read Data

In [10]:
data = pd.read_csv('chronic_kidney_disease.csv')
data

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [12]:
data['rbc'].value_counts()

rbc
normal      201
abnormal     47
Name: count, dtype: int64

In [ ]:
data = data.drop_duplicates(ignore_index=True)
data

In [ ]:
data.info()

## Data Preprocessing

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data)

# Transform the DataFrame to obtain the normalized data
data_normalized = scaler.transform(data)

In [ ]:
X = data_normalized[:, :-1]
y = data_normalized[:, -1]

## Modelling

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class MultiChannelWeightedDropout(tf.keras.Model):
    def __init__(self, out, p=0.5):
        super(MultiChannelWeightedDropout, self).__init__()

        self.in_layer = tf.keras.layers.Dense(3, activation='softmax')

        self.dropout1 = tf.keras.layers.Dropout(p)
        self.dropout2 = tf.keras.layers.Dropout(p)
        self.dropout3 = tf.keras.layers.Dropout(p)

        self.out_layer = tf.keras.layers.Dense(out, activation='sigmoid')

    def call(self, x, training=False):
        x = self.in_layer(x)
        weights = x

        # Apply weighted dropout
        channel1 = self.dropout1(x[:, 0], training=training) * weights[:, 0]
        channel2 = self.dropout2(x[:, 1], training=training) * weights[:, 1]
        channel3 = self.dropout3(x[:, 2], training=training) * weights[:, 2]

        x = tf.stack([channel1, channel2, channel3], axis=1)
        x = self.out_layer(x)
        return x

In [ ]:
def DNNModel(in_dim, out_dim):
    model = Sequential([
        tf.keras.layers.Input(shape=in_dim),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        MultiChannelWeightedDropout(out_dim, p=0.5),
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
    return model

## Cross Validation

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
fold_test = []
fold_pred = []

In [ ]:
for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    x_train, x_test, y_train, y_test = (X[train_idx]), (X[test_idx]), (y[train_idx]), (y[test_idx])
    classifier = DNNModel(8, 1)
    
    train_init = time.time()
    classifier.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
    train_time = time.time() - train_init

    test_init = time.time()
    prediction = classifier.predict(x_test)
    test_time = time.time() - test_init

    fold_test.append(y_test)
    fold_pred.append(prediction)

    preds = torch.tensor(prediction.flatten())
    trues = torch.tensor(y_test)

    print(f"Fold: {fold+1} | Accuracy: {binary_accuracy(preds=preds, target=trues).item():.5f} | Train Time: {train_time} | Test Time: {test_time}")  

In [ ]:
accuracy = []
precision = []
recall = []
f1_score = []
auroc = []

for i in range(10):
    trues = torch.tensor(fold_test[i])
    preds = torch.tensor(fold_pred[i].flatten())
    accuracy.append(binary_accuracy(preds=preds, target=trues).item())
    precision.append(binary_precision(preds=preds, target=trues).item())
    recall.append(binary_recall(preds=preds, target=trues).item())
    f1_score.append(binary_f1_score(preds=preds, target=trues).item())
    auroc.append(binary_auroc(preds.float(), trues.long()).item())

print(f"Accuracy: {np.mean(accuracy):.5f} | Precision: {np.mean(precision):.5f} | Recall: {np.mean(recall):.5f} | F1 Score: {np.mean(f1_score):.5f} | AUC ROC: {np.mean(auroc):.5f}")